# bronze DDL

```SQL
-- bronze.airbnb definition
CREATE TABLE bronze.airbnb (
	id int8 NULL,
	"name" varchar NULL,
	host_id varchar NULL,
	host_name varchar NULL,
	neighbourhood_group varchar NULL,
	neighbourhood varchar NULL,
	latitude float8 NULL,
	longitude float8 NULL,
	room_type varchar NULL,
	price float4 NULL,
	minimum_nights int4 NULL,
	number_of_reviews int4 NULL,
	last_review varchar NULL,
	reviews_per_month float4 NULL,
	calculated_host_listings_count int4 NULL,
	availability_365 int4 NULL,
	number_of_reviews_ltm int4 NULL,
	license varchar NULL
);
```


```SQL
-- bronze.cars definition
CREATE TABLE bronze.cars (
	brand varchar NULL,
	car_id int8 NULL,
	model varchar NULL,
	cylinders varchar NULL,
	transmission varchar NULL,
	drive_wheel varchar NULL,
	power varchar NULL,
	max_power_rpm varchar NULL,
	torque varchar NULL,
	max_torque_rpm varchar NULL,
	turbo varchar NULL,
	fuel varchar NULL,
	top_speed varchar NULL,
	acc_0_100 varchar NULL,
	gear_1 varchar NULL,
	gear_2 varchar NULL,
	gear_3 varchar NULL,
	gear_4 varchar NULL,
	gear_5 varchar NULL,
	gear_6 varchar NULL,
	gear_7 varchar NULL,
	gear_8 varchar NULL,
	gear_9 varchar NULL,
	gear_r varchar NULL,
	gear_final varchar NULL,
	front_tire varchar NULL,
	rear_tire varchar NULL,
	eng_capacity varchar NULL,
	weight varchar NULL,
	height varchar NULL,
	width varchar NULL,
	length varchar NULL,
	wheelbase varchar NULL
);
```


```SQL
-- silver.airbnb_room_descriptions definition
CREATE TABLE silver.airbnb_room_descriptions (
	id integer NULL,
	"name" varchar NULL,
	num_bedrooms integer NULL,
	num_beds integer NULL,
	num_bathrooms integer NULL,
	room_type varchar NULL
);
```

In [ ]:
import csv
import json
import psycopg2
import psycopg2.extras

conn_string = f"host=host dbname=golf_squad " \
              f"user=admin password=password"

cars_insert_query = """
INSERT INTO bronze.cars(
    brand, car_id, model, cylinders, transmission, drive_wheel, power, max_power_rpm, torque, max_torque_rpm,
    turbo, fuel, top_speed, acc_0_100, gear_1, gear_2, gear_3, gear_4, gear_5, gear_6, gear_7, gear_8, gear_9,
    gear_r, gear_final, front_tire, rear_tire, eng_capacity, weight, height, width, length, wheelbase
)
VALUES (
    %(brand)s, %(car_id)s, %(model)s, %(cylinders)s, %(transmission)s, %(drive_wheel)s, %(power)s, %(max_power_rpm)s,
    %(torque)s, %(max_torque_rpm)s, %(turbo)s, %(fuel)s, %(top_speed)s, %(acc_0_100)s, %(gear_1)s, %(gear_2)s, %(gear_3)s,
    %(gear_4)s, %(gear_5)s, %(gear_6)s, %(gear_7)s, %(gear_8)s, %(gear_9)s, %(gear_r)s, %(gear_final)s, %(front_tire)s,
    %(rear_tire)s, %(eng_capacity)s, %(weight)s, %(height)s, %(width)s, %(length)s, %(wheelbase)s
)
"""

airbnb_insert_query = """
INSERT INTO bronze.airbnb(
    id, name, host_id, host_name, neighbourhood_group, neighbourhood, latitude, longitude, room_type, price,
    minimum_nights, number_of_reviews, last_review, reviews_per_month, calculated_host_listings_count,
    availability_365, number_of_reviews_ltm, license
)
VALUES (
    %(id)s, %(name)s, %(host_id)s, %(host_name)s, %(neighbourhood_group)s, %(neighbourhood)s, %(latitude)s, %(longitude)s,
    %(room_type)s, %(price)s, %(minimum_nights)s, %(number_of_reviews)s, %(last_review)s, %(reviews_per_month)s,
    %(calculated_host_listings_count)s, %(availability_365)s, %(number_of_reviews_ltm)s, %(license)s
)
"""


In [ ]:
with open('input/cars.json') as json_file:
    data = json.load(json_file)

with psycopg2.connect(conn_string) as conn:
    cursor = conn.cursor()
    psycopg2.extras.execute_batch(cursor, cars_insert_query, data[:20000])
    conn.commit()

In [ ]:
with open('input/airbnb.csv') as csv_file:
    reader = csv.DictReader(csv_file)
    data = [row for row in reader]

with psycopg2.connect(conn_string) as conn:
    cursor = conn.cursor()
    psycopg2.extras.execute_batch(cursor, airbnb_insert_query, data)
    conn.commit()

In [ ]:
airbnb_sample_etl_query = """
INSERT INTO silver.airbnb_room_descriptions
(id, "name", num_bedrooms, num_beds, num_bathrooms, room_type)
SELECT
	CAST(id AS int8) AS id,
	SPLIT_PART("name", ' · ', 1)  AS "name",
	CAST(COALESCE(SUBSTRING("name", '([0-9]+) bedroom(?:s)?'), '0') AS integer) AS num_bedrooms,
	CAST(SUBSTRING("name", '([0-9]+) bed(?:s)?') AS integer) AS num_beds,
	CAST(COALESCE(SUBSTRING("name", '([0-9]+) (?:shared |private )?bath(?:s)?'), '0') AS integer) AS num_bathrooms,
	room_type
FROM bronze.airbnb
"""

with psycopg2.connect(conn_string) as conn:
    cursor = conn.cursor()
    cursor.execute(airbnb_sample_etl_query)
    conn.commit()